# 🚀 **Daily Data Pipeline - Execution Guide**
## Bronze to Silver Layer Processing with Data Quality

---

## 🎯 **PRODUCTION WORKFLOW - Execute in This Order**

### **Phase 1: Configuration** ⏱️ ~5 seconds

**Cell 1 - Configuration (Auto-discover databases and tables)**
* Scans `bronze/mysql/` to discover all databases automatically
* Auto-discovers all tables within each database
* Creates `all_database_configs` variable with paths
* **Output:** 2 databases (retail_db, students_db), 8 tables total

---

### **Phase 2: Data Quality Analysis** ⏱️ ~30 seconds

**Cell 2 - Analyze MySQL Bronze Data Quality**
* Checks all MySQL tables for nulls, duplicates, data types
* Displays sample records and schema
* Generates quality report
* **Issues Found:** 34 duplicates, null values in several tables

**Cell 3 - Analyze Event Hub Bronze Data Quality**
* Analyzes Event Hub AVRO data structure
* Checks Body field (JSON content), SequenceNumber, Offset
* Shows load date distribution

---

### **Phase 3: Data Cleaning** ⏱️ ~1 minute

**Cell 4 - Define Cleaning Rules**
* Global rules: remove duplicates, trim strings, handle nulls
* Table-specific rules: email validation, required fields, dedupe keys

**Cell 5 - Apply Cleaning to MySQL Data**
* Removes 34 duplicates (27% reduction)
* Trims strings, filters invalid emails, removes null required fields
* Creates temp views: `cleaned_{db_name}_{table_name}`
* **Results:** 124 → 90 records

**Cell 6 - Apply Cleaning to Event Hub Data**
* Removes duplicates, parses JSON Body field
* Decodes base64 content, flattens nested JSON
* Creates temp view: `cleaned_eventhub_events`

---

### **Phase 4: Validation** ⏱️ ~20 seconds

**Cell 7 - Validate Cleaned Data Quality**
* Verifies no nulls in required fields
* Confirms no duplicates remain
* **Result:** ✅ 8/8 tables passed validation

---

### **Phase 5: Event Hub Processing** ⏱️ ~1 minute

**Cell 8 - Event Hub → Bronze**
* Reads AVRO files from `streamingingestionsathya/eventhub/`
* Uses Auto Loader with `recursiveFileLookup`
* Processes nested folders: `partition/year/month/day/hour/minute`
* Writes to `bronze/eventhub/eventhub_events/` as Delta
* Partitions by load_date, uses checkpoint

**Cell 9 - Event Hub Bronze → Silver**
* Reads from bronze Event Hub Delta table
* Adds processing metadata
* Writes to `silver/eventhub/eventhub_events/`

---

### **Phase 6: MySQL to Silver** ⏱️ ~30 seconds

**Cell 10 - MySQL Cleaned Data → Silver**
* Reads CLEANED data from temp views (Cell 5)
* Adds processing metadata
* Writes to `silver/mysql/{database}/{table}/` as Delta
* **Output:** 90 clean, validated records

---

### **Phase 7: Verification** ⏱️ ~10 seconds

**Cell 11 - Complete Silver Layer Summary**
* Verifies all MySQL databases and Event Hub in silver
* Shows record counts per table
* **Final:** 3 sources, 9 tables, 92 total records

---

## ⏱️ **TOTAL DURATION: ~4 minutes**

---

## 📅 **SCHEDULING RECOMMENDATIONS**

### **Option 1: Daily Batch Processing** ⭐ RECOMMENDED

**Schedule:** Daily at 2:00 AM  
**Cron:** `0 2 * * *`  
**Run Cells:** 1 → 2 → 3 → 4 → 5 → 6 → 7 → 8 → 9 → 10 → 11

**Best For:**
* Standard daily data processing
* Both MySQL and Event Hub updated once per day
* Cost-effective (runs once per day)

**Databricks Workflow Setup:**
```
Task 1: Configuration & Cleaning
  Cells: 1, 2, 3, 4, 5, 6, 7
  Duration: ~2 min
  Dependencies: None

Task 2: Event Hub Processing
  Cells: 8, 9
  Duration: ~1 min
  Dependencies: Task 1

Task 3: MySQL to Silver
  Cell: 10
  Duration: ~30 sec
  Dependencies: Task 1

Task 4: Verification
  Cell: 11
  Duration: ~10 sec
  Dependencies: Task 2, Task 3
```

---

### **Option 2: Frequent Event Hub, Daily MySQL**

**Event Hub Schedule:** Every hour  
**Cron:** `0 * * * *`  
**Run Cells:** 1 → 8 → 9 → 11

**MySQL Schedule:** Daily at 2:00 AM  
**Cron:** `0 2 * * *`  
**Run Cells:** 1 → 2 → 4 → 5 → 7 → 10 → 11

**Best For:**
* Near real-time Event Hub processing
* Batch MySQL processing
* Different SLAs for different sources

---

### **Option 3: Skip Analysis (After Initial Setup)**

**Schedule:** Daily at 2:00 AM  
**Run Cells:** 1 → 4 → 5 → 6 → 7 → 8 → 9 → 10 → 11

**Best For:**
* After initial data quality assessment
* Faster execution (skip analysis Cells 2-3)
* Stable data sources with known quality

**Note:** Run Cells 2-3 weekly/monthly to monitor quality trends

---

## 📊 **DATA QUALITY RESULTS**

### **Bronze Layer (Raw):**
* Total records: 124
* Issues: 34 duplicates, null values in multiple tables

### **Silver Layer (Cleaned):**
* Total records: 92 (90 MySQL + 2 Event Hub)
* Quality: ✅ All validations passed (8/8 tables)
* Improvement: 27% reduction in bad data

### **Specific Improvements:**
* customer_details: 9 → 6 records (3 duplicates removed)
* orders: 42 → 21 records (21 duplicates removed)
* users: 20 → 10 records (10 duplicates removed)

---

## 📍 **STORAGE LOCATIONS**

**Bronze Layer:**
* MySQL: `bronze/mysql/{database}/{table}/load_date=YYYY-MM-DD/`
* Event Hub: `bronze/eventhub/eventhub_events/load_date=YYYY-MM-DD/`

**Silver Layer:**
* MySQL: `silver/mysql/{database}/{table}/`
* Event Hub: `silver/eventhub/eventhub_events/`

**Checkpoints:**
* Event Hub to Bronze: `checkpoints/eventhub_to_bronze/eventhub_events/`
* Event Hub to Silver: `checkpoints/bronze_to_silver/eventhub/eventhub_events/`

---

## 🔑 **KEY FEATURES**

✅ **Fully Dynamic** - Auto-discovers new databases and tables  
✅ **Incremental** - Only processes new files (checkpoint-based)  
✅ **Data Quality** - Analysis, cleaning, validation built-in  
✅ **Multi-Source** - Handles MySQL + Event Hub seamlessly  
✅ **Scalable** - Add new sources without code changes  
✅ **Monitored** - Clear success/failure indicators  

---

## ⚡ **PERFORMANCE NOTES**

**First Run:**
* Duration: ~4 minutes
* Processes ALL existing data
* Creates checkpoints

**Subsequent Runs:**
* Duration: ~30 seconds (if no new data)
* Only processes NEW files
* Fast checkpoint lookup

**Auto Loader Behavior:**
* Tracks files by path/name only (not content)
* Modified files with same name are SKIPPED
* Recommendation: Use append-only pattern with unique filenames

---

## 📝 **NOTEBOOK PATH**

`/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup`

---

## 📧 **CONTACT**

**Owner:** sathyarajeshpk@gmail.com  
**Workspace:** datamigrationsathya (Azure)  
**Compute:** Serverless Interactive Cluster

In [0]:
# ============================================
# FULLY DYNAMIC CONFIGURATION
# Auto-discovers databases AND tables
# ============================================

# Base storage account and container
storage_account = "datamigrationsathya"
container = "datalake"

# Path structure configuration
source_system = "mysql"  # mysql, postgres, etc.

# Base path for the source system
source_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{source_system}/"

print("=" * 70)
print("AUTO-DISCOVERING DATABASES AND TABLES")
print("=" * 70)
print(f"\nSource system: {source_system}")
print(f"Base path: {source_base_path}")

# ============================================
# AUTO-DISCOVER ALL DATABASES
# ============================================

all_database_configs = []

try:
    # List all database folders under bronze/mysql/
    database_folders = dbutils.fs.ls(source_base_path)
    
    print(f"\nFound {len(database_folders)} database(s):\n")
    
    for db_folder in database_folders:
        if db_folder.isDir():
            database_name = db_folder.name.rstrip('/')
            
            # Construct paths for this database
            bronze_path = f"{source_base_path}{database_name}/"
            silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/{source_system}/{database_name}/"
            checkpoint_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/bronze_to_silver/{source_system}/{database_name}/"
            
            # Auto-discover tables in this database
            try:
                table_folders = dbutils.fs.ls(bronze_path)
                tables = [t.name.rstrip('/') for t in table_folders if t.isDir()]
                
                if tables:  # Only add if tables exist
                    all_database_configs.append({
                        "database_name": database_name,
                        "bronze_path": bronze_path,
                        "silver_path": silver_path,
                        "checkpoint_path": checkpoint_path,
                        "tables": tables
                    })
                    
                    print(f"  ✓ {database_name}: {len(tables)} table(s)")
                    for table in tables:
                        print(f"      - {table}")
                    print()
                else:
                    print(f"  ⚠ {database_name}: No tables found (skipping)\n")
                    
            except Exception as e:
                print(f"  ✗ {database_name}: Error reading tables - {str(e)}\n")
                continue
    
    if not all_database_configs:
        print("\n⚠️  No databases with tables found!")
        print("Please check your bronze layer structure.")
    else:
        print("=" * 70)
        print(f"SUMMARY: {len(all_database_configs)} database(s) ready to process")
        total_tables = sum(len(config['tables']) for config in all_database_configs)
        print(f"Total tables across all databases: {total_tables}")
        print("=" * 70)
        
except Exception as e:
    print(f"\n❌ Error discovering databases: {str(e)}")
    print("\nFalling back to manual configuration...")
    
    # Fallback: Manual configuration
    database_name = "retail_db"
    bronze_base_path = f"{source_base_path}{database_name}/"
    silver_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/{source_system}/{database_name}/"
    checkpoint_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/bronze_to_silver/{source_system}/{database_name}/"
    
    # Auto-discover tables
    try:
        folders = dbutils.fs.ls(bronze_base_path)
        tables_to_process = [folder.name.rstrip('/') for folder in folders if folder.isDir()]
        print(f"Using manual database: {database_name}")
        print(f"Found {len(tables_to_process)} tables: {tables_to_process}")
    except:
        tables_to_process = ["customer_details"]
        print(f"Using fallback table list: {tables_to_process}")
    
    # Create single database config for backward compatibility
    all_database_configs = [{
        "database_name": database_name,
        "bronze_path": bronze_base_path,
        "silver_path": silver_base_path,
        "checkpoint_path": checkpoint_base_path,
        "tables": tables_to_process
    }]

print("\n✅ Configuration complete!")
print("\nNote: Auto Loader will recursively process all files in subdirectories (e.g., load_date partitions)")

In [0]:
# Comprehensive data quality analysis for MySQL bronze layer

from pyspark.sql.functions import col, count, when, isnan, isnull, sum as spark_sum, avg, min as spark_min, max as spark_max

print("=" * 70)
print("MYSQL BRONZE LAYER - DATA QUALITY ANALYSIS")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 6 (Configuration) first!")
else:
    quality_report = []
    
    for config in all_database_configs:
        db_name = config['database_name']
        bronze_path = config['bronze_path']
        tables = config['tables']
        
        print(f"\n{'='*70}")
        print(f"Database: {db_name}")
        print(f"{'='*70}")
        
        for table_name in tables:
            print(f"\n📊 Analyzing: {table_name}")
            print("-" * 70)
            
            try:
                # Read bronze data
                df = spark.read.parquet(f"{bronze_path}{table_name}/")
                
                total_records = df.count()
                total_columns = len(df.columns)
                
                print(f"  Total Records: {total_records:,}")
                print(f"  Total Columns: {total_columns}")
                
                # 1. NULL ANALYSIS
                print(f"\n  📋 Null Analysis:")
                null_counts = df.select([spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]).collect()[0]
                
                has_nulls = False
                for col_name in df.columns:
                    null_count = null_counts[col_name]
                    if null_count > 0:
                        null_pct = (null_count / total_records) * 100
                        print(f"    ⚠ {col_name}: {null_count:,} nulls ({null_pct:.1f}%)")
                        has_nulls = True
                
                if not has_nulls:
                    print(f"    ✓ No null values found")
                
                # 2. DUPLICATE ANALYSIS
                print(f"\n  🔄 Duplicate Analysis:")
                duplicate_count = total_records - df.dropDuplicates().count()
                if duplicate_count > 0:
                    print(f"    ⚠ Found {duplicate_count:,} duplicate records ({(duplicate_count/total_records)*100:.1f}%)")
                else:
                    print(f"    ✓ No duplicates found")
                
                # 3. DATA TYPE ANALYSIS
                print(f"\n  📝 Schema:")
                for field in df.schema.fields:
                    print(f"    - {field.name}: {field.dataType}")
                
                # 4. SAMPLE DATA
                print(f"\n  📄 Sample Records (first 3):")
                display(df.limit(3))
                
                # Store quality metrics
                quality_report.append({
                    'database': db_name,
                    'table': table_name,
                    'total_records': total_records,
                    'total_columns': total_columns,
                    'has_nulls': has_nulls,
                    'duplicate_count': duplicate_count
                })
                
            except Exception as e:
                print(f"  ❌ Error analyzing {table_name}: {str(e)[:100]}")
    
    # Summary Report
    print(f"\n\n{'='*70}")
    print("QUALITY ANALYSIS SUMMARY")
    print(f"{'='*70}")
    
    for report in quality_report:
        status = "⚠ Issues Found" if (report['has_nulls'] or report['duplicate_count'] > 0) else "✓ Clean"
        print(f"\n{report['database']}.{report['table']}: {status}")
        print(f"  Records: {report['total_records']:,}")
        if report['has_nulls']:
            print(f"  ⚠ Has null values")
        if report['duplicate_count'] > 0:
            print(f"  ⚠ Has {report['duplicate_count']:,} duplicates")

In [0]:
# Data quality analysis for Event Hub bronze layer

from pyspark.sql.functions import col, count, when, length, size

print("=" * 70)
print("EVENT HUB BRONZE LAYER - DATA QUALITY ANALYSIS")
print("=" * 70)

eventhub_name = "eventhub_events"
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"

try:
    df_eh = spark.read.format("delta").load(bronze_eventhub_path)
    
    total_records = df_eh.count()
    print(f"\nTotal Records: {total_records:,}")
    print(f"Total Columns: {len(df_eh.columns)}")
    
    # 1. NULL ANALYSIS
    print(f"\n📋 Null Analysis:")
    null_counts = df_eh.select([spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_eh.columns]).collect()[0]
    
    for col_name in df_eh.columns:
        null_count = null_counts[col_name]
        if null_count > 0:
            null_pct = (null_count / total_records) * 100
            print(f"  ⚠ {col_name}: {null_count:,} nulls ({null_pct:.1f}%)")
    
    # 2. BODY FIELD ANALYSIS (contains JSON data)
    print(f"\n📦 Body Field Analysis:")
    if 'Body' in df_eh.columns:
        # Check for empty bodies
        empty_bodies = df_eh.filter(col('Body').isNull()).count()
        print(f"  Empty bodies: {empty_bodies}")
        
        # Sample body content
        print(f"\n  Sample Body content (first record):")
        sample_body = df_eh.select('Body').limit(1).collect()[0]['Body']
        if sample_body:
            import base64
            decoded = base64.b64decode(sample_body).decode('utf-8')
            print(f"  {decoded[:200]}...")
    
    # 3. DUPLICATE ANALYSIS
    print(f"\n🔄 Duplicate Analysis:")
    duplicate_count = total_records - df_eh.dropDuplicates(['SequenceNumber', 'Offset']).count()
    if duplicate_count > 0:
        print(f"  ⚠ Found {duplicate_count:,} duplicate records")
    else:
        print(f"  ✓ No duplicates found")
    
    # 4. LOAD DATE DISTRIBUTION
    print(f"\n📅 Load Date Distribution:")
    if 'load_date' in df_eh.columns:
        df_eh.groupBy('load_date').count().orderBy('load_date').show()
    
    # 5. SCHEMA
    print(f"\n📝 Schema:")
    df_eh.printSchema()
    
    # 6. SAMPLE DATA
    print(f"\n📄 Sample Records:")
    display(df_eh.limit(3))
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Make sure Event Hub data has been copied to bronze layer (run Cell 20)")

In [0]:
# Define data cleaning rules and transformations

from pyspark.sql.functions import col, trim, upper, lower, regexp_replace, to_timestamp, coalesce, lit

print("=" * 70)
print("DATA CLEANING RULES DEFINITION")
print("=" * 70)

# ============================================
# CLEANING RULES CONFIGURATION
# ============================================

cleaning_rules = {
    # Global rules applied to all tables
    'global': {
        'remove_duplicates': True,
        'trim_strings': True,
        'handle_nulls': True,
        'standardize_dates': True
    },
    
    # Table-specific rules
    'table_specific': {
        'customer_details': {
            'required_fields': ['customer_id', 'customer_name'],
            'email_validation': True,
            'dedupe_key': ['customer_id']
        },
        'orders': {
            'required_fields': ['order_id'],
            'dedupe_key': ['order_id'],
            'filter_negative_amounts': True
        },
        'users': {
            'required_fields': ['user_id', 'name'],
            'dedupe_key': ['user_id']
        },
        'eventhub_events': {
            'required_fields': ['SequenceNumber', 'Offset'],
            'dedupe_key': ['SequenceNumber', 'Offset'],
            'parse_body_json': True
        }
    }
}

print("\n✅ Cleaning rules defined:")
print(f"\n📋 Global Rules:")
for rule, enabled in cleaning_rules['global'].items():
    print(f"  - {rule}: {enabled}")

print(f"\n📋 Table-Specific Rules:")
for table, rules in cleaning_rules['table_specific'].items():
    print(f"\n  {table}:")
    for rule, value in rules.items():
        print(f"    - {rule}: {value}")

print("\n" + "=" * 70)
print("Rules ready to apply!")
print("=" * 70)

In [0]:
# Apply cleaning transformations to MySQL bronze data

from pyspark.sql.functions import col, trim, regexp_replace, when, length, coalesce

def clean_mysql_table(df, table_name, rules):
    """
    Apply cleaning transformations to a DataFrame
    """
    df_clean = df
    
    # Get table-specific rules
    table_rules = rules['table_specific'].get(table_name, {})
    
    # 1. REMOVE DUPLICATES
    if rules['global']['remove_duplicates']:
        dedupe_key = table_rules.get('dedupe_key', None)
        if dedupe_key:
            initial_count = df_clean.count()
            df_clean = df_clean.dropDuplicates(dedupe_key)
            removed = initial_count - df_clean.count()
            if removed > 0:
                print(f"    ✓ Removed {removed:,} duplicates based on {dedupe_key}")
    
    # 2. TRIM STRING COLUMNS
    if rules['global']['trim_strings']:
        string_cols = [field.name for field in df_clean.schema.fields if str(field.dataType) == 'StringType']
        for col_name in string_cols:
            df_clean = df_clean.withColumn(col_name, trim(col(col_name)))
        if string_cols:
            print(f"    ✓ Trimmed {len(string_cols)} string columns")
    
    # 3. FILTER REQUIRED FIELDS
    required_fields = table_rules.get('required_fields', [])
    if required_fields:
        initial_count = df_clean.count()
        for field in required_fields:
            if field in df_clean.columns:
                df_clean = df_clean.filter(col(field).isNotNull())
        removed = initial_count - df_clean.count()
        if removed > 0:
            print(f"    ✓ Filtered {removed:,} records with null required fields")
    
    # 4. EMAIL VALIDATION (if applicable)
    if table_rules.get('email_validation', False):
        email_cols = [c for c in df_clean.columns if 'email' in c.lower()]
        for email_col in email_cols:
            initial_count = df_clean.count()
            df_clean = df_clean.filter(
                (col(email_col).isNull()) | 
                (col(email_col).rlike(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'))
            )
            removed = initial_count - df_clean.count()
            if removed > 0:
                print(f"    ✓ Filtered {removed:,} records with invalid emails")
    
    # 5. FILTER NEGATIVE AMOUNTS (if applicable)
    if table_rules.get('filter_negative_amounts', False):
        amount_cols = [c for c in df_clean.columns if 'amount' in c.lower() or 'price' in c.lower()]
        for amount_col in amount_cols:
            if amount_col in df_clean.columns:
                initial_count = df_clean.count()
                df_clean = df_clean.filter((col(amount_col).isNull()) | (col(amount_col) >= 0))
                removed = initial_count - df_clean.count()
                if removed > 0:
                    print(f"    ✓ Filtered {removed:,} records with negative {amount_col}")
    
    return df_clean


print("=" * 70)
print("APPLYING CLEANING TRANSFORMATIONS - MYSQL")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 6 (Configuration) first!")
else:
    cleaning_summary = []
    
    for config in all_database_configs:
        db_name = config['database_name']
        bronze_path = config['bronze_path']
        tables = config['tables']
        
        print(f"\n{'='*70}")
        print(f"Database: {db_name}")
        print(f"{'='*70}")
        
        for table_name in tables:
            print(f"\n  🧹 Cleaning: {table_name}")
            
            try:
                # Read bronze data
                df_bronze = spark.read.parquet(f"{bronze_path}{table_name}/")
                initial_count = df_bronze.count()
                
                # Apply cleaning
                df_clean = clean_mysql_table(df_bronze, table_name, cleaning_rules)
                final_count = df_clean.count()
                
                removed = initial_count - final_count
                retention_pct = (final_count / initial_count) * 100 if initial_count > 0 else 0
                
                print(f"    📊 Initial: {initial_count:,} → Final: {final_count:,} ({retention_pct:.1f}% retained)")
                
                # Store cleaned DataFrame for later use
                df_clean.createOrReplaceTempView(f"cleaned_{db_name}_{table_name}")
                
                cleaning_summary.append({
                    'database': db_name,
                    'table': table_name,
                    'initial': initial_count,
                    'final': final_count,
                    'removed': removed
                })
                
            except Exception as e:
                print(f"    ❌ Error: {str(e)[:100]}")
    
    # Summary
    print(f"\n\n{'='*70}")
    print("CLEANING SUMMARY")
    print(f"{'='*70}")
    
    total_initial = sum(s['initial'] for s in cleaning_summary)
    total_final = sum(s['final'] for s in cleaning_summary)
    total_removed = sum(s['removed'] for s in cleaning_summary)
    
    for summary in cleaning_summary:
        print(f"\n{summary['database']}.{summary['table']}:")
        print(f"  Initial: {summary['initial']:,}")
        print(f"  Final: {summary['final']:,}")
        print(f"  Removed: {summary['removed']:,}")
    
    print(f"\n{'='*70}")
    print(f"TOTAL: {total_initial:,} → {total_final:,} (removed {total_removed:,})")
    print(f"{'='*70}")

In [0]:
# Apply cleaning transformations to Event Hub bronze data

from pyspark.sql.functions import col, from_json, schema_of_json, base64, unbase64
import json

print("=" * 70)
print("APPLYING CLEANING TRANSFORMATIONS - EVENT HUB")
print("=" * 70)

eventhub_name = "eventhub_events"
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"

try:
    df_eh = spark.read.format("delta").load(bronze_eventhub_path)
    initial_count = df_eh.count()
    
    print(f"\nInitial records: {initial_count:,}")
    
    # 1. REMOVE DUPLICATES
    print(f"\n🧹 Cleaning steps:")
    df_clean = df_eh.dropDuplicates(['SequenceNumber', 'Offset'])
    removed_dupes = initial_count - df_clean.count()
    if removed_dupes > 0:
        print(f"  ✓ Removed {removed_dupes:,} duplicates")
    else:
        print(f"  ✓ No duplicates found")
    
    # 2. FILTER NULL REQUIRED FIELDS
    df_clean = df_clean.filter(
        col('SequenceNumber').isNotNull() & 
        col('Offset').isNotNull()
    )
    removed_nulls = df_eh.count() - df_clean.count() - removed_dupes
    if removed_nulls > 0:
        print(f"  ✓ Filtered {removed_nulls:,} records with null required fields")
    
    # 3. PARSE BODY JSON (if Body field exists)
    if 'Body' in df_clean.columns:
        print(f"\n  📦 Parsing Body JSON field...")
        
        # Decode base64 Body to string
        from pyspark.sql.functions import expr
        df_clean = df_clean.withColumn('body_string', expr("cast(unbase64(Body) as string)"))
        
        # Try to infer JSON schema from sample
        sample_body = df_clean.select('body_string').filter(col('body_string').isNotNull()).limit(1).collect()
        
        if sample_body and sample_body[0]['body_string']:
            try:
                # Infer schema from sample JSON
                json_schema = schema_of_json(sample_body[0]['body_string'])
                
                # Parse JSON
                df_clean = df_clean.withColumn('parsed_body', from_json(col('body_string'), json_schema))
                
                # Flatten parsed JSON fields
                if 'parsed_body' in df_clean.columns:
                    # Get all fields from parsed_body struct
                    parsed_fields = df_clean.select('parsed_body.*').columns
                    for field in parsed_fields:
                        df_clean = df_clean.withColumn(f"body_{field}", col(f"parsed_body.{field}"))
                    
                    print(f"    ✓ Parsed Body JSON into {len(parsed_fields)} fields")
            except Exception as e:
                print(f"    ⚠ Could not parse JSON: {str(e)[:100]}")
    
    final_count = df_clean.count()
    retention_pct = (final_count / initial_count) * 100 if initial_count > 0 else 0
    
    print(f"\n📊 Summary:")
    print(f"  Initial: {initial_count:,}")
    print(f"  Final: {final_count:,}")
    print(f"  Removed: {initial_count - final_count:,}")
    print(f"  Retention: {retention_pct:.1f}%")
    
    # Store cleaned DataFrame
    df_clean.createOrReplaceTempView("cleaned_eventhub_events")
    
    print(f"\n✅ Event Hub data cleaned and ready!")
    print(f"\n📄 Sample cleaned data:")
    display(df_clean.limit(5))
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Make sure Event Hub data has been copied to bronze layer (run Cell 20)")

In [0]:
# Validate cleaned data quality

print("=" * 70)
print("DATA QUALITY VALIDATION - POST CLEANING")
print("=" * 70)

validation_results = []

# Validate MySQL tables
if 'all_database_configs' in dir() and all_database_configs:
    for config in all_database_configs:
        db_name = config['database_name']
        tables = config['tables']
        
        for table_name in tables:
            view_name = f"cleaned_{db_name}_{table_name}"
            
            try:
                df = spark.table(view_name)
                
                # Check for nulls in required fields
                table_rules = cleaning_rules['table_specific'].get(table_name, {})
                required_fields = table_rules.get('required_fields', [])
                
                null_in_required = False
                for field in required_fields:
                    if field in df.columns:
                        null_count = df.filter(col(field).isNull()).count()
                        if null_count > 0:
                            null_in_required = True
                            break
                
                # Check for duplicates
                dedupe_key = table_rules.get('dedupe_key', [])
                has_duplicates = False
                if dedupe_key:
                    total = df.count()
                    unique = df.dropDuplicates(dedupe_key).count()
                    has_duplicates = (total != unique)
                
                status = "✅ PASS" if (not null_in_required and not has_duplicates) else "⚠️ ISSUES"
                
                validation_results.append({
                    'table': f"{db_name}.{table_name}",
                    'status': status,
                    'null_in_required': null_in_required,
                    'has_duplicates': has_duplicates,
                    'record_count': df.count()
                })
                
            except Exception as e:
                validation_results.append({
                    'table': f"{db_name}.{table_name}",
                    'status': "❌ ERROR",
                    'error': str(e)[:50]
                })

# Validate Event Hub
try:
    df_eh = spark.table("cleaned_eventhub_events")
    
    null_in_required = df_eh.filter(
        col('SequenceNumber').isNull() | col('Offset').isNull()
    ).count() > 0
    
    has_duplicates = df_eh.count() != df_eh.dropDuplicates(['SequenceNumber', 'Offset']).count()
    
    status = "✅ PASS" if (not null_in_required and not has_duplicates) else "⚠️ ISSUES"
    
    validation_results.append({
        'table': 'eventhub.eventhub_events',
        'status': status,
        'null_in_required': null_in_required,
        'has_duplicates': has_duplicates,
        'record_count': df_eh.count()
    })
except:
    pass

# Display results
print("\n📋 Validation Results:\n")
for result in validation_results:
    print(f"{result['status']} {result['table']}")
    if 'record_count' in result:
        print(f"    Records: {result['record_count']:,}")
    if result.get('null_in_required'):
        print(f"    ⚠ Has nulls in required fields")
    if result.get('has_duplicates'):
        print(f"    ⚠ Has duplicate records")
    if 'error' in result:
        print(f"    ❌ Error: {result['error']}")
    print()

print("=" * 70)
passed = sum(1 for r in validation_results if r['status'] == "✅ PASS")
total = len(validation_results)
print(f"Validation: {passed}/{total} tables passed")
print("=" * 70)

In [0]:
# ============================================
# PRODUCTION: Write CLEANED MySQL data to Silver Layer
# This replaces the old Cell 19 - now uses cleaned data
# ============================================

from pyspark.sql.functions import current_timestamp, lit

print("=" * 70)
print("MYSQL CLEANED DATA TO SILVER PIPELINE")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ ERROR: Please run Cell 6 first!")
else:
    print(f"\n✅ Found configuration for {len(all_database_configs)} database(s)\n")
    
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\n{'='*70}")
        print(f"Database: {db_name}")
        print(f"{'='*70}")
        
        for table_name in tables:
            print(f"\n  📊 Processing: {db_name}/{table_name}")
            
            try:
                # Read CLEANED data from temp view
                view_name = f"cleaned_{db_name}_{table_name}"
                df_clean = spark.table(view_name)
                
                # Add processing metadata
                df_silver = (df_clean
                    .withColumn("processing_timestamp", current_timestamp())
                    .withColumn("source_table", lit(table_name))
                    .withColumn("source_database", lit(db_name))
                )
                
                # Write to silver layer (overwrite mode for cleaned data)
                df_silver.write\
                    .format("delta")\
                    .mode("overwrite")\
                    .option("mergeSchema", "true")\
                    .save(f"{silver_path}{table_name}/")
                
                record_count = df_silver.count()
                print(f"     ✅ Written {record_count:,} cleaned records to silver")
                
            except Exception as e:
                print(f"     ❌ Error: {str(e)[:100]}")
                continue
        
        print(f"\n  ✅ Completed database: {db_name}")
    
    print("\n" + "="*70)
    print("✅ MySQL Cleaned Data Pipeline Complete!")
    print("="*70)

In [0]:
# ============================================
# DAILY PRODUCTION PIPELINE
# Processes all databases and tables discovered in Cell 1
# ============================================

from pyspark.sql.functions import current_timestamp, lit

print("=" * 70)
print("Starting Daily Bronze to Silver Pipeline")
print("=" * 70)

# Use the all_database_configs from Cell 1
if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ ERROR: Please run Cell 1 first to discover databases and tables!")
    print("Cell 1 creates the 'all_database_configs' variable needed for processing.")
else:
    print(f"\n✅ Found configuration for {len(all_database_configs)} database(s)")
    total_tables = sum(len(config['tables']) for config in all_database_configs)
    print(f"Total tables to process: {total_tables}\n")
    
    # Process each database
    for config in all_database_configs:
        db_name = config['database_name']
        bronze_path = config['bronze_path']
        silver_path = config['silver_path']
        checkpoint_path = config['checkpoint_path']
        tables = config['tables']
        
        print(f"\n{'='*70}")
        print(f"Database: {db_name}")
        print(f"Tables: {len(tables)}")
        print(f"{'='*70}")
        
        # Process each table in this database
        for table_name in tables:
            print(f"\n  📊 Processing: {db_name}/{table_name}")
            
            try:
                # Read with Auto Loader (only processes new files)
                df_stream = (spark.readStream
                    .format("cloudFiles")
                    .option("cloudFiles.format", "parquet")
                    .option("cloudFiles.schemaLocation", f"{checkpoint_path}{table_name}/schema")
                    .option("cloudFiles.inferColumnTypes", "true")
                    .option("recursiveFileLookup", "true")
                    .load(f"{bronze_path}{table_name}/")
                )
                
                # Add metadata columns
                df_enriched = (df_stream
                    .withColumn("processing_timestamp", current_timestamp())
                    .withColumn("source_table", lit(table_name))
                    .withColumn("source_database", lit(db_name))
                )
                
                # Write to silver layer
                query = (df_enriched.writeStream
                    .format("delta")
                    .option("checkpointLocation", f"{checkpoint_path}{table_name}/checkpoint")
                    .option("mergeSchema", "true")
                    .outputMode("append")
                    .trigger(availableNow=True)
                    .start(f"{silver_path}{table_name}/")
                )
                
                # Wait for completion
                query.awaitTermination()
                
                print(f"     ✅ Successfully processed {table_name}")
                
            except Exception as e:
                print(f"     ❌ Error processing {table_name}: {str(e)[:100]}")
                continue
        
        print(f"\n  ✅ Completed database: {db_name}")
    
    print("\n" + "="*70)
    print("✅ Daily Pipeline Complete!")
    print("="*70)

In [0]:
# ============================================
# EVENT HUB TO BRONZE PIPELINE
# Run this to copy new Event Hub data to bronze layer
# ============================================

from pyspark.sql.functions import current_timestamp, lit, to_date

print("=" * 70)
print("EVENT HUB TO BRONZE PIPELINE")
print("=" * 70)

# Configuration
eventhub_source = f"abfss://{container}@{storage_account}.dfs.core.windows.net/streamingingestionsathya/eventhub/"
eventhub_name = "eventhub_events"

# Bronze destination
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"
checkpoint_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/eventhub_to_bronze/{eventhub_name}/"

print(f"\nSource: {eventhub_source}")
print(f"Destination: {bronze_eventhub_path}")
print(f"\n⏳ Processing Event Hub data...\n")

try:
    # Read Event Hub data with Auto Loader
    df_eventhub = (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "avro")  # Event Hub Capture uses AVRO
        .option("cloudFiles.schemaLocation", f"{checkpoint_eventhub_path}schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("recursiveFileLookup", "true")
        .load(eventhub_source)
    )
    
    # Add metadata and partition by date
    df_enriched = (df_eventhub
        .withColumn("ingestion_timestamp", current_timestamp())
        .withColumn("source_system", lit("eventhub"))
        .withColumn("load_date", to_date(current_timestamp()))
    )
    
    # Write to bronze layer
    query = (df_enriched.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{checkpoint_eventhub_path}checkpoint")
        .option("mergeSchema", "true")
        .partitionBy("load_date")
        .trigger(availableNow=True)
        .start(bronze_eventhub_path)
    )
    
    query.awaitTermination()
    
    print("\n" + "=" * 70)
    print("✅ Event Hub data copied to Bronze layer!")
    print("=" * 70)
    
    # Verify
    df_bronze = spark.read.format("delta").load(bronze_eventhub_path)
    record_count = df_bronze.count()
    
    print(f"\nTotal records in bronze: {record_count:,}")
    
    if "load_date" in df_bronze.columns:
        dates = df_bronze.select("load_date").distinct().count()
        print(f"Distinct load dates: {dates}")
    
except Exception as e:
    print(f"\n❌ Error: {str(e)[:100]}")
    import traceback
    traceback.print_exc()

In [0]:
# ============================================
# EVENT HUB BRONZE TO SILVER PIPELINE
# Run this after copying Event Hub data to bronze
# ============================================

from pyspark.sql.functions import current_timestamp, lit

print("=" * 70)
print("EVENT HUB BRONZE TO SILVER PIPELINE")
print("=" * 70)

# Configuration
eventhub_name = "eventhub_events"
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"
silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"
checkpoint_silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/bronze_to_silver/eventhub/{eventhub_name}/"

print(f"\nBronze: {bronze_eventhub_path}")
print(f"Silver: {silver_eventhub_path}")
print(f"\n⏳ Processing Event Hub bronze to silver...\n")

try:
    # Read from bronze layer
    df_bronze = (spark.readStream
        .format("delta")
        .load(bronze_eventhub_path)
    )
    
    # Add processing metadata
    df_enriched = (df_bronze
        .withColumn("processing_timestamp", current_timestamp())
        .withColumn("source_table", lit(eventhub_name))
        .withColumn("source_database", lit("eventhub"))
    )
    
    # Write to silver layer
    query = (df_enriched.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{checkpoint_silver_path}checkpoint")
        .option("mergeSchema", "true")
        .trigger(availableNow=True)
        .start(silver_eventhub_path)
    )
    
    query.awaitTermination()
    
    print("\n" + "=" * 70)
    print("✅ Event Hub data processed to Silver layer!")
    print("=" * 70)
    
    # Verify
    df_silver = spark.read.format("delta").load(silver_eventhub_path)
    record_count = df_silver.count()
    
    print(f"\nTotal records in silver: {record_count:,}")
    
    if "load_date" in df_silver.columns:
        dates = df_silver.select("load_date").distinct().count()
        print(f"Distinct load dates: {dates}")
    
except Exception as e:
    print(f"\n❌ Error: {str(e)[:100]}")
    import traceback
    traceback.print_exc()

In [0]:
# Complete verification of all data sources in silver layer

print("=" * 70)
print("COMPLETE SILVER LAYER SUMMARY")
print("=" * 70)

total_sources = 0
total_tables = 0
total_records = 0

# 1. MySQL Databases
print("\n📊 MYSQL DATABASES")
print("=" * 70)

if 'all_database_configs' in dir() and all_database_configs:
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\n  Database: {db_name}")
        
        db_records = 0
        for table_name in tables:
            try:
                df = spark.read.format("delta").load(f"{silver_path}{table_name}/")
                count = df.count()
                db_records += count
                print(f"    ✓ {table_name}: {count:,} records")
            except Exception as e:
                print(f"    ✗ {table_name}: Not processed")
        
        print(f"  Subtotal: {db_records:,} records")
        total_sources += 1
        total_tables += len(tables)
        total_records += db_records
else:
    print("  ⚠ No MySQL databases configured")

# 2. Event Hub
print("\n\n📡 EVENT HUB")
print("=" * 70)

eventhub_name = "eventhub_events"
silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"

try:
    df_eh = spark.read.format("delta").load(silver_eventhub_path)
    eh_count = df_eh.count()
    
    print(f"\n  Source: eventhub")
    print(f"    ✓ {eventhub_name}: {eh_count:,} records")
    
    if "load_date" in df_eh.columns:
        dates = df_eh.select("load_date").distinct().count()
        print(f"    ✓ Load dates: {dates}")
    
    print(f"  Subtotal: {eh_count:,} records")
    total_sources += 1
    total_tables += 1
    total_records += eh_count
    
except Exception as e:
    print(f"\n  ✗ Event Hub: Not processed or error")
    print(f"     {str(e)[:80]}")

# Final Summary
print("\n\n" + "=" * 70)
print("FINAL SUMMARY")
print("=" * 70)
print(f"\n  Total Sources: {total_sources}")
print(f"  Total Tables/Streams: {total_tables}")
print(f"  Total Records: {total_records:,}")
print("\n" + "=" * 70)

if total_records > 0:
    print("\n✅ SUCCESS! All data sources processed to silver layer!")
    print("\n📍 Silver Layer Locations:")
    print(f"   - MySQL: abfss://{container}@{storage_account}.dfs.core.windows.net/silver/mysql/")
    print(f"   - Event Hub: abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/")
else:
    print("\n⚠️ No data found in silver layer. Please run the processing pipelines first.")

---
# ⚡ **Delta Lake Optimization**

## **Why Optimize?**

* **Query Performance** - Up to 10-100x faster
* **Storage Efficiency** - 30-50% reduction
* **Cost Savings** - Less storage + faster queries

---

## **Techniques**

### **1. OPTIMIZE** - File Compaction
* Combines small files into larger files
* Run weekly after incremental loads

### **2. Z-ORDER** - Data Clustering  
* Co-locates related data
* Enables data skipping
* Best for 2-4 columns

### **3. Liquid Clustering** ⭐ RECOMMENDED
* Automatic incremental clustering
* Self-optimizing
* Best for 3-5 columns

### **4. Optimized Writes**
* Auto-compacts during writes
* Enable at session start

### **5. VACUUM**
* Removes old file versions
* Run monthly
* Saves 20-50% storage

In [0]:
# Compact small files in Silver layer

print("=" * 70)
print("OPTIMIZING SILVER LAYER - FILE COMPACTION")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 2 (Configuration) first!")
else:
    # Optimize MySQL tables
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\nDatabase: {db_name}")
        
        for table_name in tables:
            try:
                table_path = f"{silver_path}{table_name}/"
                spark.sql(f"OPTIMIZE delta.`{table_path}`")
                print(f"  ✅ {table_name}: Optimized")
            except Exception as e:
                print(f"  ❌ {table_name}: {str(e)[:80]}")
    
    # Optimize Event Hub
    eventhub_name = "eventhub_events"
    silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"
    
    try:
        spark.sql(f"OPTIMIZE delta.`{silver_eventhub_path}`")
        print(f"\n  ✅ eventhub/{eventhub_name}: Optimized")
    except Exception as e:
        print(f"\n  ❌ eventhub/{eventhub_name}: {str(e)[:80]}")
    
    print(f"\n{'='*70}")
    print("✅ OPTIMIZE COMPLETE")
    print(f"{'='*70}")

In [0]:
# Apply Z-ORDER clustering for query performance

print("=" * 70)
print("APPLYING Z-ORDER CLUSTERING")
print("=" * 70)

# Z-ORDER columns for each table
zorder_config = {
    'customer_details': ['customer_id', 'load_date'],
    'customer_orders': ['order_id', 'customer_id'],
    'orders': ['order_id', 'user_id'],
    'users': ['user_id', 'country'],
    'monthly_active_users': ['year__of_month'],
    'departments': ['department_id'],
    'employees': ['employee_id', 'department_id'],
    'students_table': ['id', 'grade'],
    'eventhub_events': ['load_date', 'SequenceNumber']
}

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 2 first!")
else:
    # Z-ORDER MySQL tables
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\nDatabase: {db_name}")
        
        for table_name in tables:
            if table_name in zorder_config:
                zorder_cols = zorder_config[table_name]
                try:
                    table_path = f"{silver_path}{table_name}/"
                    zorder_clause = ', '.join(zorder_cols)
                    spark.sql(f"OPTIMIZE delta.`{table_path}` ZORDER BY ({zorder_clause})")
                    print(f"  ✅ {table_name}: Z-ORDERed by {', '.join(zorder_cols)}")
                except Exception as e:
                    print(f"  ❌ {table_name}: {str(e)[:80]}")
    
    # Z-ORDER Event Hub
    eventhub_name = "eventhub_events"
    if eventhub_name in zorder_config:
        zorder_cols = zorder_config[eventhub_name]
        silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"
        try:
            zorder_clause = ', '.join(zorder_cols)
            spark.sql(f"OPTIMIZE delta.`{silver_eventhub_path}` ZORDER BY ({zorder_clause})")
            print(f"\n  ✅ eventhub/{eventhub_name}: Z-ORDERed by {', '.join(zorder_cols)}")
        except Exception as e:
            print(f"\n  ❌ eventhub/{eventhub_name}: {str(e)[:80]}")
    
    print(f"\n{'='*70}")
    print("✅ Z-ORDER COMPLETE")
    print(f"{'='*70}")

In [0]:
# Enable Liquid Clustering (DBR 13.3+)

print("=" * 70)
print("ENABLING LIQUID CLUSTERING")
print("=" * 70)

clustering_config = {
    'customer_details': ['customer_id', 'load_date'],
    'customer_orders': ['customer_id', 'order_id', 'load_date'],
    'orders': ['user_id', 'order_date', 'load_date'],
    'users': ['user_id', 'country', 'load_date'],
    'monthly_active_users': ['year__of_month', 'load_date'],
    'departments': ['department_id', 'load_date'],
    'employees': ['department_id', 'employee_id', 'load_date'],
    'students_table': ['grade', 'id', 'load_date'],
    'eventhub_events': ['load_date', 'SequenceNumber']
}

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 2 first!")
else:
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\nDatabase: {db_name}")
        
        for table_name in tables:
            if table_name in clustering_config:
                cluster_cols = clustering_config[table_name]
                try:
                    table_path = f"{silver_path}{table_name}/"
                    cluster_clause = ', '.join(cluster_cols)
                    spark.sql(f"ALTER TABLE delta.`{table_path}` CLUSTER BY ({cluster_clause})")
                    print(f"  ✅ {table_name}: Liquid Clustering enabled")
                except Exception as e:
                    if "CLUSTER BY is only supported" in str(e):
                        print(f"  ⚠️  {table_name}: Not supported (use Z-ORDER instead)")
                    else:
                        print(f"  ❌ {table_name}: {str(e)[:80]}")
    
    # Event Hub
    eventhub_name = "eventhub_events"
    if eventhub_name in clustering_config:
        cluster_cols = clustering_config[eventhub_name]
        silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"
        try:
            cluster_clause = ', '.join(cluster_cols)
            spark.sql(f"ALTER TABLE delta.`{silver_eventhub_path}` CLUSTER BY ({cluster_clause})")
            print(f"\n  ✅ eventhub/{eventhub_name}: Liquid Clustering enabled")
        except Exception as e:
            if "CLUSTER BY is only supported" in str(e):
                print(f"\n  ⚠️  eventhub/{eventhub_name}: Not supported (use Z-ORDER)")
            else:
                print(f"\n  ❌ eventhub/{eventhub_name}: {str(e)[:80]}")
    
    print(f"\n{'='*70}")
    print("✅ LIQUID CLUSTERING COMPLETE")
    print(f"{'='*70}")

In [0]:
# Enable Optimized Writes for better write performance

print("=" * 70)
print("ENABLING OPTIMIZED WRITES")
print("=" * 70)

# Try to enable optimized writes
try:
    spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
    print("✅ spark.databricks.delta.optimizeWrite.enabled = true")
except Exception as e:
    print("⚠️  spark.databricks.delta.optimizeWrite.enabled: Not available on Serverless")
    print("   (Serverless has built-in write optimizations)")

# Try to enable auto compaction
try:
    spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
    print("✅ spark.databricks.delta.autoCompact.enabled = true")
except Exception as e:
    print("⚠️  spark.databricks.delta.autoCompact.enabled: Not available on Serverless")
    print("   (Serverless has built-in auto compaction)")

print("\n" + "=" * 70)
print("✅ WRITE OPTIMIZATION CONFIGURED")
print("=" * 70)
print("\n💡 Note: Serverless compute has built-in write optimizations")
print("   These settings are only needed on classic clusters")

In [0]:
# VACUUM removes old file versions (Run monthly)
# WARNING: Permanently deletes old versions

print("=" * 70)
print("VACUUM - CLEAN UP OLD FILE VERSIONS")
print("=" * 70)

print("\n⚠️  WARNING: Code is commented for safety")
print("   Uncomment to execute\n")

retention_hours = 168  # 7 days

# UNCOMMENT TO RUN:
# if 'all_database_configs' in dir() and all_database_configs:
#     for config in all_database_configs:
#         db_name = config['database_name']
#         silver_path = config['silver_path']
#         for table_name in config['tables']:
#             try:
#                 table_path = f"{silver_path}{table_name}/"
#                 spark.sql(f"VACUUM delta.`{table_path}` RETAIN {retention_hours} HOURS")
#                 print(f"  ✅ {db_name}.{table_name}: Vacuumed")
#             except Exception as e:
#                 print(f"  ❌ {db_name}.{table_name}: {str(e)[:80]}")
#     
#     # Event Hub
#     eventhub_name = "eventhub_events"
#     silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"
#     try:
#         spark.sql(f"VACUUM delta.`{silver_eventhub_path}` RETAIN {retention_hours} HOURS")
#         print(f"\n  ✅ eventhub/{eventhub_name}: Vacuumed")
#     except Exception as e:
#         print(f"\n  ❌ eventhub/{eventhub_name}: {str(e)[:80]}")

print("\n💡 Uncomment code above to run VACUUM")

# 📊 **Change Data Feed (CDF)**

## **What is Change Data Feed?**

CDF tracks **row-level changes** in Delta tables:
* **INSERT** - New records added
* **UPDATE** - Existing records modified
* **DELETE** - Records removed
* **Metadata:** _change_type, _commit_version, _commit_timestamp

---

## **Why Enable CDF on Bronze Layer?**

### **Benefits:**
* **Track source system changes** - See what data arrived when
* **Audit trail** - Complete history of data ingestion
* **Incremental Silver processing** - Process only new Bronze data
* **Data lineage** - Track data from source to Silver
* **Debugging** - Identify when bad data entered the system

### **Use Cases:**
* Incremental Bronze → Silver processing (instead of full table scans)
* Audit compliance (track all data received)
* Data quality monitoring (detect anomalies)
* Replay specific changes

---

## **Why Enable CDF on Silver Layer?**

### **Benefits:**
* **Incremental Gold layer** - Process only changed Silver data
* **Downstream consumers** - Stream changes to data warehouse, BI tools
* **Real-time analytics** - Update dashboards incrementally
* **Data synchronization** - Keep systems in sync

### **Use Cases:**
* Incremental aggregations for Gold layer
* Real-time dashboard updates
* Sync to Snowflake, Synapse, etc.
* Event-driven architectures

---

## **Performance Impact**

* **Storage:** +5-10% (stores change metadata)
* **Write performance:** <5% slower
* **Read performance:** No impact on normal queries
* **Change queries:** Very fast (only reads changes)

---

## **Recommendation**

✅ **Enable CDF on BOTH Bronze and Silver layers**

**Why?**
* Enables end-to-end change tracking
* Future-proofs your pipeline
* Minimal overhead
* Easy to enable now, hard to add later
* Only tracks changes AFTER enablement

---

## **Architecture with CDF**

```
Source Systems
    ↓
Bronze Layer (CDF enabled)
    ↓ Track: What data arrived
    ↓ Process: Only new Bronze data
Silver Layer (CDF enabled)
    ↓ Track: What data was cleaned/transformed
    ↓ Process: Only changed Silver data
Gold Layer
    ↓ Incremental aggregations
Dashboards/Analytics
```

In [0]:
# Enable Change Data Feed on Bronze layer MySQL tables

print("=" * 70)
print("ENABLING CDF ON BRONZE LAYER - MYSQL")
print("=" * 70)

print("\n📊 Why enable CDF on Bronze:")
print("  • Track what data arrived from source systems")
print("  • Enable incremental Bronze → Silver processing")
print("  • Audit trail of all ingested data")
print("  • Detect data quality issues at source\n")

if 'all_database_configs' not in dir() or not all_database_configs:
    print("❌ Please run Cell 2 (Configuration) first!")
else:
    enabled_count = 0
    failed_count = 0
    
    for config in all_database_configs:
        db_name = config['database_name']
        bronze_path = config['bronze_path']
        tables = config['tables']
        
        print(f"\nDatabase: {db_name}")
        print("-" * 70)
        
        for table_name in tables:
            try:
                table_path = f"{bronze_path}{table_name}/"
                
                # Enable Change Data Feed
                spark.sql(f"""
                    ALTER TABLE delta.`{table_path}`
                    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
                """)
                
                print(f"  ✅ {table_name}: CDF enabled")
                enabled_count += 1
                
            except Exception as e:
                print(f"  ❌ {table_name}: {str(e)[:80]}")
                failed_count += 1
    
    print(f"\n{'='*70}")
    print("BRONZE LAYER CDF SUMMARY")
    print(f"{'='*70}")
    print(f"\n✅ Enabled: {enabled_count} tables")
    if failed_count > 0:
        print(f"❌ Failed: {failed_count} tables")
    print(f"\n{'='*70}")

In [0]:
# Enable Change Data Feed on Bronze layer Event Hub table

print("=" * 70)
print("ENABLING CDF ON BRONZE LAYER - EVENT HUB")
print("=" * 70)

eventhub_name = "eventhub_events"
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"

print(f"\nTable: eventhub/{eventhub_name}")
print(f"Path: {bronze_eventhub_path}\n")

try:
    # Enable Change Data Feed
    spark.sql(f"""
        ALTER TABLE delta.`{bronze_eventhub_path}`
        SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
    """)
    
    print("✅ CDF enabled on Event Hub Bronze layer")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

print(f"\n{'='*70}")
print("EVENT HUB BRONZE CDF ENABLED")
print(f"{'='*70}")

In [0]:
# Enable Change Data Feed on Silver layer tables

print("=" * 70)
print("ENABLING CDF ON SILVER LAYER")
print("=" * 70)

print("\n📊 Benefits:")
print("  • Incremental Gold layer processing")
print("  • Stream changes to downstream systems")
print("  • Audit trail of transformations")
print("  • Real-time analytics\n")

if 'all_database_configs' not in dir() or not all_database_configs:
    print("❌ Please run Cell 2 first!")
else:
    enabled_count = 0
    failed_count = 0
    
    # Enable CDF for MySQL Silver tables
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\nDatabase: {db_name}")
        print("-" * 70)
        
        for table_name in tables:
            try:
                table_path = f"{silver_path}{table_name}/"
                
                spark.sql(f"""
                    ALTER TABLE delta.`{table_path}`
                    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
                """)
                
                print(f"  ✅ {table_name}: CDF enabled")
                enabled_count += 1
                
            except Exception as e:
                print(f"  ❌ {table_name}: {str(e)[:80]}")
                failed_count += 1
    
    # Enable CDF for Event Hub Silver
    print(f"\nEvent Hub")
    print("-" * 70)
    
    eventhub_name = "eventhub_events"
    silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"
    
    try:
        spark.sql(f"""
            ALTER TABLE delta.`{silver_eventhub_path}`
            SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
        """)
        
        print(f"  ✅ {eventhub_name}: CDF enabled")
        enabled_count += 1
        
    except Exception as e:
        print(f"  ❌ {eventhub_name}: {str(e)[:80]}")
        failed_count += 1
    
    # Summary
    print(f"\n{'='*70}")
    print("SILVER LAYER CDF SUMMARY")
    print(f"{'='*70}")
    print(f"\n✅ Enabled: {enabled_count} tables")
    if failed_count > 0:
        print(f"❌ Failed: {failed_count} tables")
    print(f"\n{'='*70}")

In [0]:
# Verify CDF is enabled and show usage examples

from pyspark.sql.functions import col

print("=" * 70)
print("CHANGE DATA FEED - VERIFICATION & EXAMPLES")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 2 first!")
else:
    # Check CDF status on a sample table
    config = all_database_configs[0]
    db_name = config['database_name']
    bronze_path = config['bronze_path']
    silver_path = config['silver_path']
    table_name = config['tables'][0]
    
    print(f"\n📊 Sample table: {db_name}.{table_name}\n")
    
    # Check Bronze layer CDF
    print("=" * 70)
    print("BRONZE LAYER CDF STATUS")
    print("=" * 70)
    
    bronze_table_path = f"{bronze_path}{table_name}/"
    
    try:
        # Check table properties
        props = spark.sql(f"DESCRIBE DETAIL delta.`{bronze_table_path}`").collect()[0]
        
        # CDF is enabled if property exists
        print(f"\nBronze table: {bronze_table_path}")
        print(f"✅ CDF should be enabled (run Cell 21 if not)\n")
        
        # Try to read changes
        df_bronze_changes = spark.read \
            .format("delta") \
            .option("readChangeFeed", "true") \
            .option("startingVersion", 0) \
            .load(bronze_table_path)
        
        change_count = df_bronze_changes.count()
        print(f"Changes tracked in Bronze: {change_count}")
        
        if change_count > 0:
            print("\nChange types:")
            df_bronze_changes.groupBy("_change_type").count().show()
        
    except Exception as e:
        print(f"⚠️  {str(e)[:100]}")
        print("Note: CDF only tracks changes AFTER it's enabled")
    
    # Check Silver layer CDF
    print("\n" + "=" * 70)
    print("SILVER LAYER CDF STATUS")
    print("=" * 70)
    
    silver_table_path = f"{silver_path}{table_name}/"
    
    try:
        print(f"\nSilver table: {silver_table_path}")
        print(f"✅ CDF should be enabled (run Cell 23 if not)\n")
        
        # Try to read changes
        df_silver_changes = spark.read \
            .format("delta") \
            .option("readChangeFeed", "true") \
            .option("startingVersion", 0) \
            .load(silver_table_path)
        
        change_count = df_silver_changes.count()
        print(f"Changes tracked in Silver: {change_count}")
        
        if change_count > 0:
            print("\nChange types:")
            df_silver_changes.groupBy("_change_type").count().show()
        
    except Exception as e:
        print(f"⚠️  {str(e)[:100]}")
        print("Note: CDF only tracks changes AFTER it's enabled")
    
    # Usage examples
    print("\n" + "=" * 70)
    print("USAGE EXAMPLES")
    print("=" * 70)
    
    print("""
    
1. Read changes from Bronze (for incremental Silver processing):

df_new_bronze = spark.read \\
    .format("delta") \\
    .option("readChangeFeed", "true") \\
    .option("startingVersion", last_processed_version) \\
    .load(bronze_path)

# Process only new data to Silver
df_new_bronze.filter(col("_change_type") == "insert").write...


2. Read changes from Silver (for incremental Gold processing):

df_new_silver = spark.read \\
    .format("delta") \\
    .option("readChangeFeed", "true") \\
    .option("startingTimestamp", "2026-02-01") \\
    .load(silver_path)

# Build Gold layer incrementally
df_new_silver.groupBy("customer_id").agg(...).write...


3. Stream changes in real-time:

df_stream = spark.readStream \\
    .format("delta") \\
    .option("readChangeFeed", "true") \\
    .load(silver_path)

# Process changes as they arrive
df_stream.writeStream.format("delta").start(gold_path)
    """)
    
    print("\n" + "=" * 70)
    print("💡 CDF enables powerful incremental processing patterns!")
    print("=" * 70)

In [0]:
# Enable Change Data Feed on Bronze layer MySQL tables

print("=" * 70)
print("ENABLING CDF ON BRONZE LAYER - MYSQL")
print("=" * 70)

print("\n📊 Enabling CDF on Bronze layer...\n")

if 'all_database_configs' not in dir() or not all_database_configs:
    print("❌ Please run Cell 2 (Configuration) first!")
else:
    enabled_count = 0
    failed_count = 0
    
    for config in all_database_configs:
        db_name = config['database_name']
        bronze_path = config['bronze_path']
        tables = config['tables']
        
        print(f"Database: {db_name}")
        print("-" * 70)
        
        for table_name in tables:
            try:
                table_path = f"{bronze_path}{table_name}/"
                
                # Enable Change Data Feed
                spark.sql(f"""
                    ALTER TABLE delta.`{table_path}`
                    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
                """)
                
                print(f"  ✅ {table_name}: CDF enabled")
                enabled_count += 1
                
            except Exception as e:
                print(f"  ❌ {table_name}: {str(e)[:80]}")
                failed_count += 1
        
        print()
    
    print("=" * 70)
    print("BRONZE LAYER (MySQL) CDF SUMMARY")
    print("=" * 70)
    print(f"\n✅ Enabled: {enabled_count} tables")
    if failed_count > 0:
        print(f"❌ Failed: {failed_count} tables")
    print(f"\n{'='*70}")
    print("💡 CDF will track all future changes to Bronze layer!")
    print(f"{'='*70}")

In [0]:
# Enable Change Data Feed on Bronze layer Event Hub table

print("=" * 70)
print("ENABLING CDF ON BRONZE LAYER - EVENT HUB")
print("=" * 70)

eventhub_name = "eventhub_events"
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"

print(f"\nTable: eventhub/{eventhub_name}")
print(f"Path: {bronze_eventhub_path}\n")

try:
    # Enable Change Data Feed
    spark.sql(f"""
        ALTER TABLE delta.`{bronze_eventhub_path}`
        SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
    """)
    
    print("✅ CDF enabled on Event Hub Bronze layer")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

print(f"\n{'='*70}")
print("EVENT HUB BRONZE CDF ENABLED")
print(f"{'='*70}")
print("\n💡 CDF will track all Event Hub data arriving in Bronze!")

In [0]:
# Enable Change Data Feed on Silver layer tables

print("=" * 70)
print("ENABLING CDF ON SILVER LAYER")
print("=" * 70)

print("\n📊 Enabling CDF on Silver layer...\n")

if 'all_database_configs' not in dir() or not all_database_configs:
    print("❌ Please run Cell 2 first!")
else:
    enabled_count = 0
    failed_count = 0
    
    # Enable CDF for MySQL Silver tables
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"Database: {db_name}")
        print("-" * 70)
        
        for table_name in tables:
            try:
                table_path = f"{silver_path}{table_name}/"
                
                spark.sql(f"""
                    ALTER TABLE delta.`{table_path}`
                    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
                """)
                
                print(f"  ✅ {table_name}: CDF enabled")
                enabled_count += 1
                
            except Exception as e:
                print(f"  ❌ {table_name}: {str(e)[:80]}")
                failed_count += 1
        
        print()
    
    # Enable CDF for Event Hub Silver
    print(f"Event Hub")
    print("-" * 70)
    
    eventhub_name = "eventhub_events"
    silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"
    
    try:
        spark.sql(f"""
            ALTER TABLE delta.`{silver_eventhub_path}`
            SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
        """)
        
        print(f"  ✅ {eventhub_name}: CDF enabled")
        enabled_count += 1
        
    except Exception as e:
        print(f"  ❌ {eventhub_name}: {str(e)[:80]}")
        failed_count += 1
    
    print(f"\n{'='*70}")
    print("SILVER LAYER CDF SUMMARY")
    print(f"{'='*70}")
    print(f"\n✅ Enabled: {enabled_count} tables")
    if failed_count > 0:
        print(f"❌ Failed: {failed_count} tables")
    print(f"\n{'='*70}")
    print("💡 CDF is now tracking all changes in Bronze AND Silver!")
    print(f"{'='*70}")
    
    print("\n📋 Summary:")
    print("  • Bronze layer: Tracks data ingestion from sources")
    print("  • Silver layer: Tracks cleaned/transformed data")
    print("  • Ready for incremental Gold layer processing")
    print("  • Complete audit trail from source to Silver")

---
# ✅ **Change Data Feed - Complete**

---

## 📊 **What Was Enabled**

### **Bronze Layer CDF:**
* **MySQL tables:** 8 tables (retail_db + students_db)
* **Event Hub:** 1 table (eventhub_events)
* **Total:** 9 Bronze tables

### **Silver Layer CDF:**
* **MySQL tables:** 8 tables
* **Event Hub:** 1 table
* **Total:** 9 Silver tables

### **Grand Total:** 18 tables with CDF enabled

---

## 🎯 **What This Enables**

### **1. Incremental Bronze → Silver Processing**
```python
# Instead of processing entire Bronze table:
df_bronze = spark.read.format("delta").load(bronze_path)

# Process only NEW data:
df_new = spark.read \\
    .format("delta") \\
    .option("readChangeFeed", "true") \\
    .option("startingVersion", last_version) \\
    .load(bronze_path)
```

**Benefits:** 10-100x faster, lower costs

### **2. Incremental Silver → Gold Processing**
```python
# Process only changed Silver data:
df_changes = spark.read \\
    .format("delta") \\
    .option("readChangeFeed", "true") \\
    .option("startingTimestamp", "2026-02-01") \\
    .load(silver_path)

# Build Gold layer incrementally
df_changes.groupBy("customer_id").agg(...).write...
```

**Benefits:** Real-time aggregations, faster Gold layer

### **3. Audit Trail**
```python
# See what changed and when:
df_changes.select(
    "_change_type",
    "_commit_version", 
    "_commit_timestamp",
    "customer_id",
    "customer_name"
).show()
```

**Benefits:** Compliance, debugging, data lineage

### **4. Streaming Changes**
```python
# Stream changes to downstream systems:
df_stream = spark.readStream \\
    .format("delta") \\
    .option("readChangeFeed", "true") \\
    .load(silver_path)

df_stream.writeStream \\
    .format("delta") \\
    .start(gold_path)
```

**Benefits:** Real-time analytics, event-driven architecture

---

## 📅 **Updated Daily Workflow**

### **With CDF Enabled:**

**Daily Run:**
```
Cells 2-13 (Data processing)
```

**Future Gold Layer (Incremental):**
```
# Only process changes since last run
df_changes = spark.read \\
    .option("readChangeFeed", "true") \\
    .option("startingTimestamp", last_run_time) \\
    .load(silver_path)
```

---

## 💰 **Cost Savings with CDF**

### **Without CDF:**
* Process entire Silver table every time
* Example: 1M records × daily = 1M records processed

### **With CDF:**
* Process only changed records
* Example: 10K new records × daily = 10K records processed
* **Savings: 99% reduction in data processed!**

---

## 🔧 **Maintenance**

### **Storage Impact:**
* Bronze layer: +5-10% storage
* Silver layer: +5-10% storage
* Total overhead: Minimal

### **VACUUM Consideration:**
* CDF data is stored in Delta log
* VACUUM removes old versions but keeps CDF metadata
* Retention period: 30 days (default)

---

## ✅ **Next Steps**

1. **CDF is now enabled** on Bronze and Silver layers
2. **Future changes will be tracked** automatically
3. **Build Gold layer** using incremental CDF queries
4. **Monitor CDF overhead** (should be minimal)
5. **Use CDF for audit queries** when needed

---

## 📚 **Documentation**

For more on CDF, see:
* Databricks docs: Change Data Feed
* Use `table_changes()` SQL function
* Query with `readChangeFeed` option